In [1]:
import os
from glob import glob
import numpy as np
import pandas as pd
import xarray as xr
import geopandas as gpd
from shapely.geometry import box
from scipy.special import erf
import matplotlib.pyplot as plt

In [2]:
CLIMATE_DIR = "climate"
CURR_KEY = "20201201-20301130"
FUT_KEY  = "20701201-20801130"


RAIL_SHP = "railshape/NWR_TrackCentreLines.shp"
OLE_SHP  = "oleshape/oleshape.shp"


OUT_DIR = "outputs_final_with_shapes5"
os.makedirs(OUT_DIR, exist_ok=True)


OPEN_WITH_DASK = True
TIME_CHUNK = 360

THR = 0.05


RAIL_MU0, RAIL_SIG = 34.0, 2.5
OLE_MU0,  OLE_SIG  = 33.0, 2.0


RAIL_MU1, OLE_MU1 = 41.5, 53.0


RAIL_TON_PER_KM = 1.96
OLE_TON_PER_KM  = 0.957

In [3]:
def open_period_nc(climate_dir: str, keyword: str) -> xr.Dataset:

    files = sorted([f for f in glob(os.path.join(climate_dir, "*.nc")) if keyword in os.path.basename(f)])
    if not files:
        raise FileNotFoundError(f"not found {keyword}  .nc files")

    ds_list = []
    for fp in files:
        ds = xr.open_dataset(fp, decode_times=False)

        var = None
        for name in ['tasmax','tas','TXx','t2m','temperature','air_temperature']:
            if name in ds.data_vars and 'time' in ds[name].dims and ds[name].ndim >= 3:
                var = name; break
        if var is None:
            for name, v in ds.data_vars.items():
                if v.ndim >= 3 and 'time' in v.dims:
                    var = name; break
        if var is None:
            raise ValueError(f"{os.path.basename(fp)} notfound2")

        v = ds[var]

        rename_map = {}
        for d in v.dims:
            dl = d.lower()
            if dl.startswith('ensemble'):
                rename_map[d] = 'member'
            elif 'projection_y' in dl or d == 'y':
                rename_map[d] = 'y'
            elif 'projection_x' in dl or d == 'x':
                rename_map[d] = 'x'
            elif d == 'time':
                pass
        v = v.rename(rename_map)
        if 'member' not in v.dims:
            v = v.expand_dims('member')

        dsn = xr.Dataset()
        dsn['temp'] = v.astype('float32')
        units = v.attrs.get('units','').lower()
        if units in ['k','kelvin']:
            dsn['temp'] = dsn['temp'] - 273.15
        dsn['temp'].attrs['units'] = 'degC'


        if 'projection_x_coordinate' in ds.coords:
            dsn = dsn.assign_coords(x=ds['projection_x_coordinate'].rename({'projection_x_coordinate':'x'}))
        if 'projection_y_coordinate' in ds.coords:
            dsn = dsn.assign_coords(y=ds['projection_y_coordinate'].rename({'projection_y_coordinate':'y'}))
        if 'projection_x_coordinate_bnds' in ds.variables:
            dsn['x_bnds'] = ds['projection_x_coordinate_bnds'].rename({'projection_x_coordinate':'x'})
        if 'projection_y_coordinate_bnds' in ds.variables:
            dsn['y_bnds'] = ds['projection_y_coordinate_bnds'].rename({'projection_y_coordinate':'y'})


        if 'grid_latitude' in ds.variables:
            lat = ds['grid_latitude']
            if lat.ndim == 2:
                lat = lat.rename({lat.dims[-2]: 'y', lat.dims[-1]: 'x'})
            dsn = dsn.assign_coords(lat=lat)

        if 'grid_longitude' in ds.variables:
            lon = ds['grid_longitude']
            if lon.ndim == 2:
                lon = lon.rename({lon.dims[-2]: 'y', lon.dims[-1]: 'x'}) 
            dsn = dsn.assign_coords(lon=lon)



        for tname in ['yyyymmdd','year','month_number']:
            if tname in ds.variables:
                dsn[tname] = ds[tname]

        # grid mapping
        if 'transverse_mercator' in ds.variables:
            dsn['transverse_mercator'] = ds['transverse_mercator']

        ds_list.append(dsn)

    ds_all = xr.concat(ds_list, dim='time')
    if OPEN_WITH_DASK:
        ds_all = ds_all.chunk({'time': TIME_CHUNK})
    return ds_all


def grid_polygons_from_bounds(ds: xr.Dataset):
    if 'x_bnds' not in ds or 'y_bnds' not in ds:
        raise ValueError("notfound x_bnds / y_bnds；ensure nc have projection_*_bnds")
    xb = ds['x_bnds']; yb = ds['y_bnds']
    if 'time' in xb.dims: xb = xb.isel(time=0)
    if 'time' in yb.dims: yb = yb.isel(time=0)

    x_b = np.asarray(xb)  # (Nx,2)
    y_b = np.asarray(yb)  # (Ny,2)
    Nx = x_b.shape[0]; Ny = y_b.shape[0]

    rows, cols, geoms = [], [], []
    for i in range(Ny):
        y0, y1 = float(y_b[i,0]), float(y_b[i,1]); ys, yn = (y0, y1) if y0 <= y1 else (y1, y0)
        for j in range(Nx):
            x0, x1 = float(x_b[j,0]), float(x_b[j,1]); xw, xe = (x0, x1) if x0 <= x1 else (x1, x0)
            geoms.append(box(xw, ys, xe, yn))
            rows.append(i); cols.append(j)
    grid_gdf = gpd.GeoDataFrame({'row': rows, 'col': cols}, geometry=geoms, crs="EPSG:27700")
    return grid_gdf, (Ny, Nx)


def select_cells_with_internal_line(grid_gdf: gpd.GeoDataFrame, line_shp: str):
    lines = gpd.read_file(line_shp).to_crs(27700)
    if len(lines) == 0:
        raise ValueError(f"{os.path.basename(line_shp)} no items")
    sindex = lines.sindex

    keep = []
    for idx, cell in grid_gdf.iterrows():
        cand = list(sindex.intersection(cell.geometry.bounds))
        if not cand:
            continue
        inter = lines.iloc[cand].intersection(cell.geometry)
        total_len = 0.0
        for geom in inter:
            if not geom.is_empty:
                total_len += getattr(geom, "length", 0.0)
        if total_len > 0:
            keep.append(idx)

    sel = grid_gdf.iloc[keep].copy()
    mask_idx = pd.MultiIndex.from_arrays([sel['row'].values, sel['col'].values], names=['row','col'])
    return sel, mask_idx


def make_selected_mask(ds: xr.Dataset, mask_idx: pd.MultiIndex):
    Ny, Nx = ds.sizes['y'], ds.sizes['x']
    rows = np.asarray(mask_idx.get_level_values('row'))
    cols = np.asarray(mask_idx.get_level_values('col'))
    mask_np = np.zeros((Ny, Nx), dtype=bool)
    mask_np[rows, cols] = True
    mask = xr.DataArray(mask_np, coords={'y': ds['y'], 'x': ds['x']}, dims=('y','x'))
    return mask, rows, cols


def daily_mean_over_selected_cells(ds: xr.Dataset, mask_idx: pd.MultiIndex):
    mask, _, _ = make_selected_mask(ds, mask_idx)
    temp = ds['temp'].isel(member=0)  # (time,y,x)
    return temp.where(mask).mean(dim=('y','x'), skipna=True)


def pick_extreme_days(daily_mean: xr.DataArray):
    imax = int(daily_mean.argmax('day').values)
    imin = int(daily_mean.argmin('day').values)
    vmax = float(daily_mean.isel(day=imax).values)
    vmin = float(daily_mean.isel(day=imin).values)
    med_val = float(np.median(daily_mean.values))
    diff = np.abs(daily_mean.values - med_val)
    imed = int(np.argmin(diff))
    vmed = float(daily_mean.isel(day=imed).values)
    return {'max': {'day': imax, 'value': vmax},
            'min': {'day': imin, 'value': vmin},
            'med': {'day': imed, 'value': vmed}}


def build_average_year(ds: xr.Dataset) -> xr.Dataset:
   
    temp = ds['temp'].isel(member=0)
    nt = temp.sizes['time']
    if nt % 360 != 0:
        raise ValueError(f"time 1 {nt} not 360 times")

    day_coord = xr.DataArray(np.arange(nt) % 360, dims='time', name='day')
    year_coord = xr.DataArray(np.arange(nt) // 360, dims='time', name='year')
    temp = temp.assign_coords(day=day_coord, year=year_coord)

    temp_avg = temp.groupby('day').mean(dim='time', skipna=True)

    out = xr.Dataset({'temp_avg': temp_avg})
    for c in ['x','y','lat','lon']:
        if c in ds.coords:
            out = out.assign_coords({c: ds[c]})
    return out


def summer_slice_from_average_year(ds_avg: xr.Dataset):
    """360 days，5–8 is day ∈ [120, 240) """
    all_days = np.arange(360)
    summer_idx = all_days[(all_days >= 120) & (all_days < 240)]
    return summer_idx


def det_pf_on_cells(temp2d: xr.DataArray, mask_idx: pd.MultiIndex, mu: float, sigma: float) -> pd.DataFrame:

    ds_tmp = temp2d.to_dataset(name='temp')
    mask, _, _ = make_selected_mask(ds_tmp, mask_idx)
    temp_cells = temp2d.where(mask).stack(cell=('y','x')).dropna('cell')


    T = temp_cells.values.astype(np.float64)  # [n_cells]
    pf = 0.5 * (1.0 + erf((T - mu) / (sigma * np.sqrt(2.0))))


    Ny, Nx = mask.shape
    flat_mask = mask.values.reshape(-1)
    flat_rows = np.repeat(np.arange(Ny), Nx)
    flat_cols = np.tile(np.arange(Nx), Ny)
    sel_flat = flat_mask.nonzero()[0]
    rows = flat_rows[sel_flat]
    cols = flat_cols[sel_flat]

    return pd.DataFrame({'row': rows, 'col': cols, 'pf': pf})


def det_pf_cellwise_mu(temp2d: xr.DataArray, mask_idx: pd.MultiIndex, enhanced_idx_set,
                       mu_enhanced: float, mu_default: float, sigma: float) -> pd.DataFrame:
  
    ds_tmp = temp2d.to_dataset(name='temp')
    mask, _, _ = make_selected_mask(ds_tmp, mask_idx)
    temp_cells = temp2d.where(mask).stack(cell=('y','x')).dropna('cell')

    T = temp_cells.values.astype(np.float64)

    Ny, Nx = mask.shape
    flat_mask = mask.values.reshape(-1)
    flat_rows = np.repeat(np.arange(Ny), Nx)
    flat_cols = np.tile(np.arange(Nx), Ny)
    sel_flat = flat_mask.nonzero()[0]
    rows = flat_rows[sel_flat]
    cols = flat_cols[sel_flat]

    enh_set = set(enhanced_idx_set) if isinstance(enhanced_idx_set, (set, list, tuple)) else set()
    is_enh = np.fromiter(((r, c) in enh_set for r, c in zip(rows, cols)), dtype=bool, count=T.size)
    mu_vec = np.where(is_enh, mu_enhanced, mu_default).astype(np.float64)

    pf = 0.5 * (1.0 + erf((T - mu_vec) / (sigma * np.sqrt(2.0))))
    return pd.DataFrame({'row': rows, 'col': cols, 'pf': pf})



def compute_length_for_cells(cells_gdf: gpd.GeoDataFrame, line_shp: str) -> pd.DataFrame:
    lines = gpd.read_file(line_shp).to_crs(27700)
    if len(lines) == 0:
        raise ValueError(f"{os.path.basename(line_shp)} no tiems")
    sindex = lines.sindex

    out = []
    for _, cell in cells_gdf.iterrows():
        cand = list(sindex.intersection(cell.geometry.bounds))
        total_len = 0.0
        if cand:
            inter = lines.iloc[cand].intersection(cell.geometry)
            for geom in inter:
                if not geom.is_empty:
                    total_len += getattr(geom, "length", 0.0)
        out.append({'row': int(cell['row']), 'col': int(cell['col']), 'length_m': float(total_len)})
    return pd.DataFrame(out)


def save_pf_maps_and_thr(rail_cells_gdf, ole_cells_gdf, pf_dict_rail, pf_dict_ole,
                         thr, rail_shp, ole_shp, save_dir, save_prefix):
    os.makedirs(save_dir, exist_ok=True)


    try:
        rail_lines = gpd.read_file(rail_shp).to_crs(27700)
    except Exception:
        rail_lines = None
    try:
        ole_lines = gpd.read_file(ole_shp).to_crs(27700)
    except Exception:
        ole_lines = None

    def _save_one(asset_cells_gdf, df_pf, title, cmap, out_png, line_gdf=None):
        plot_gdf = asset_cells_gdf.merge(df_pf[['row','col','pf']], on=['row','col'], how='left')
        fig, ax = plt.subplots(figsize=(8,10))
        plot_gdf.plot(column='pf', cmap=cmap, vmin=0, vmax=0.2, legend=True,
                      ax=ax, edgecolor='grey', linewidth=0.2,
                      missing_kwds={"color":"lightgrey"})
        if line_gdf is not None and len(line_gdf) > 0:
            line_gdf.plot(ax=ax, color='black', linewidth=0.5, alpha=0.7)
        ax.set_title(title, fontsize=12)
        ax.axis('off'); plt.tight_layout()
        fig.savefig(out_png, dpi=300)
        plt.close(fig)


    for tag in ['max','med','min']:
        if tag in pf_dict_rail and pf_dict_rail[tag] is not None:
            _save_one(rail_cells_gdf, pf_dict_rail[tag], f"RAIL PF — {tag} day",
                      "Reds", os.path.join(save_dir, f"{save_prefix}_rail_pf_{tag}.png"), rail_lines)
        if tag in pf_dict_ole and pf_dict_ole[tag] is not None:
            _save_one(ole_cells_gdf, pf_dict_ole[tag], f"OLE PF — {tag} day",
                      "Blues", os.path.join(save_dir, f"{save_prefix}_ole_pf_{tag}.png"), ole_lines)


    def _thr_plot(cells_gdf, df_pf_max, line_gdf, title, color, out_name):
        idx = df_pf_max['pf'] > thr
        thr_df = df_pf_max.loc[idx, ['row','col']]
        thr_gdf = cells_gdf.merge(thr_df, on=['row','col'], how='inner')
        fig, ax = plt.subplots(figsize=(8,10))
        cells_gdf.plot(ax=ax, color='lightgrey', edgecolor='none')
        thr_gdf.plot(ax=ax, color=color, edgecolor='black', linewidth=0.2)
        if line_gdf is not None and len(line_gdf) > 0:
            line_gdf.plot(ax=ax, color='black', linewidth=0.5, alpha=0.7)
        ax.set_title(title, fontsize=12)
        ax.axis('off'); plt.tight_layout()
        fig.savefig(os.path.join(save_dir, out_name), dpi=300)
        plt.close(fig)
        return thr_gdf

    rail_thr_gdf = _thr_plot(
        rail_cells_gdf, pf_dict_rail['max'], rail_lines,
        f"RAIL PF > {thr} (max day)", "red",
        f"{save_prefix}_rail_thr_maxday.png"
    )
    ole_thr_gdf  = _thr_plot(
        ole_cells_gdf, pf_dict_ole['max'], ole_lines,
        f"OLE PF > {thr} (max day)",  "blue",
        f"{save_prefix}_ole_thr_maxday.png"
    )
    return rail_thr_gdf, ole_thr_gdf


def sum_length_and_material(thr_cells_gdf: gpd.GeoDataFrame, line_shp: str, ton_per_km: float, label: str):
    if len(thr_cells_gdf) == 0:
        detail = pd.DataFrame(columns=['row','col','length_m','length_km','material_t'])
        summary = {'label': label, 'cells': 0, 'length_m': 0.0, 'length_km': 0.0, 'material_t': 0.0}
        return summary, detail
    len_df = compute_length_for_cells(thr_cells_gdf, line_shp)
    len_df['length_km'] = len_df['length_m'] / 1000.0
    len_df['material_t'] = len_df['length_km'] * ton_per_km
    summary = {
        'label': label,
        'cells': int(len_df.shape[0]),
        'length_m': float(len_df['length_m'].sum()),
        'length_km': float(len_df['length_km'].sum()),
        'material_t': float(len_df['material_t'].sum())
    }
    return summary, len_df[['row','col','length_m','length_km','material_t']]


def _length_km_from_thr(thr_gdf: gpd.GeoDataFrame, line_shp: str) -> float:
    if len(thr_gdf) == 0:
        return 0.0
    len_df = compute_length_for_cells(thr_gdf, line_shp)
    return float(len_df['length_m'].sum() / 1000.0)



def process_one_period(ds_period: xr.Dataset,
                        rail_cells_gdf: gpd.GeoDataFrame,
                        ole_cells_gdf: gpd.GeoDataFrame,
                        rail_mask_idx: pd.MultiIndex,
                        ole_mask_idx: pd.MultiIndex,
                        period_tag: str,
                        save_nc_path: str):
 

    ds_avg = build_average_year(ds_period)  # {'temp_avg': (day,y,x)}


    summer_idx = summer_slice_from_average_year(ds_avg)
    daily_mean = ds_avg['temp_avg'].where(make_selected_mask(ds_period, rail_mask_idx)[0]).mean(dim=('y','x'), skipna=True)
    daily_mean_summer = daily_mean.isel(day=summer_idx)
    picks_rel = pick_extreme_days(daily_mean_summer)
    picks = {tag: {'day': int(summer_idx[info['day']]), 'value': info['value']} for tag, info in picks_rel.items()}


    sel_days = [picks['max']['day'], picks['med']['day'], picks['min']['day']]
    temp_sel = ds_avg['temp_avg'].isel(day=sel_days)
    ds_save = xr.Dataset({'temp': temp_sel.rename({'day':'sel_day'})})
    ds_save = ds_save.assign_coords(sel_day=('sel_day', sel_days))
    ds_save.to_netcdf(save_nc_path)

    
    day_temps = {tag: ds_avg['temp_avg'].isel(day=info['day']) for tag, info in picks.items()}
    pf_rail = {}
    pf_ole  = {}
    for tag, t2d in day_temps.items():
        pf_rail[tag] = det_pf_on_cells(t2d, rail_mask_idx, mu=RAIL_MU0, sigma=RAIL_SIG)
        pf_ole[tag]  = det_pf_on_cells(t2d, ole_mask_idx,  mu=OLE_MU0,  sigma=OLE_SIG)


    period_dir = os.path.join(OUT_DIR, period_tag)
    os.makedirs(period_dir, exist_ok=True)
    rail_thr_gdf, ole_thr_gdf = save_pf_maps_and_thr(
        rail_cells_gdf=rail_cells_gdf,
        ole_cells_gdf=ole_cells_gdf,
        pf_dict_rail=pf_rail, pf_dict_ole=pf_ole, thr=THR,
        rail_shp=RAIL_SHP, ole_shp=OLE_SHP,
        save_dir=period_dir, save_prefix=f"{period_tag}"
    )

    return picks, day_temps, pf_rail, pf_ole, rail_thr_gdf, ole_thr_gdf

In [4]:
def main():

    ds_curr = open_period_nc(CLIMATE_DIR, CURR_KEY)
    ds_fut  = open_period_nc(CLIMATE_DIR, FUT_KEY)
    print(f"finished time={ds_curr.sizes['time']}；future time={ds_fut.sizes['time']}，space=({ds_curr.sizes['y']},{ds_curr.sizes['x']})")


    grid_gdf, _ = grid_polygons_from_bounds(ds_curr)


    rail_cells_gdf, rail_mask_idx = select_cells_with_internal_line(grid_gdf, RAIL_SHP)
    ole_cells_gdf,  ole_mask_idx  = select_cells_with_internal_line(grid_gdf, OLE_SHP)
    print(f"rail grid：{len(rail_cells_gdf)}；ole grid：{len(ole_cells_gdf)}")


    curr_nc_path = os.path.join(OUT_DIR, "current_three_days.nc")
    fut_nc_path  = os.path.join(OUT_DIR, "future_three_days.nc")

    curr_picks, curr_day_temps, pf_curr_rail, pf_curr_ole, curr_thr_rail_gdf, curr_thr_ole_gdf = \
        process_one_period(ds_curr,
                           rail_cells_gdf, ole_cells_gdf,
                           rail_mask_idx, ole_mask_idx,
                           period_tag="current",
                           save_nc_path=curr_nc_path)

    fut_picks, fut_day_temps, pf_fut_rail, pf_fut_ole, fut_thr_rail_gdf, fut_thr_ole_gdf = \
        process_one_period(ds_fut,
                           rail_cells_gdf, ole_cells_gdf,
                           rail_mask_idx, ole_mask_idx,
                           period_tag="future_pre",
                           save_nc_path=fut_nc_path)


    def _picks_to_str(tag, picks):
        return f"{tag}: max={picks['max']['day']}, med={picks['med']['day']}, min={picks['min']['day']} (day 索引 0-359)"
    print("day(summer)：")
    print("  current ->", _picks_to_str("current", curr_picks))
    print("  future ->", _picks_to_str("future", fut_picks))


    fut_len_rail_sum, fut_rail_detail = sum_length_and_material(fut_thr_rail_gdf, RAIL_SHP, RAIL_TON_PER_KM, "RAIL (future, max day)")
    fut_len_ole_sum,  fut_ole_detail  = sum_length_and_material(fut_thr_ole_gdf,  OLE_SHP,  OLE_TON_PER_KM,  "OLE (future, max day)")

    mat_df = pd.DataFrame([fut_len_rail_sum, fut_len_ole_sum])
    mat_csv = os.path.join(OUT_DIR, "materials_future_maxday.csv")
    mat_df.to_csv(mat_csv, index=False)
    if len(fut_rail_detail) > 0:
        fut_rail_detail.to_csv(os.path.join(OUT_DIR, "materials_future_maxday_rail_detail.csv"), index=False)
    if len(fut_ole_detail) > 0:
        fut_ole_detail.to_csv(os.path.join(OUT_DIR, "materials_future_maxday_ole_detail.csv"), index=False)
    print(f"material saved：{mat_csv}")


    rail_enhanced_set = set(zip(fut_thr_rail_gdf['row'].tolist(), fut_thr_rail_gdf['col'].tolist()))
    ole_enhanced_set  = set(zip(fut_thr_ole_gdf['row'].tolist(),  fut_thr_ole_gdf['col'].tolist()))

    pf_fut_post_rail = {}
    pf_fut_post_ole  = {}
    for tag in ['max','med','min']:
        pf_fut_post_rail[tag] = det_pf_cellwise_mu(
            fut_day_temps[tag], rail_mask_idx,
            enhanced_idx_set=rail_enhanced_set,
            mu_enhanced=RAIL_MU1, mu_default=RAIL_MU0,
            sigma=RAIL_SIG
        )
        pf_fut_post_ole[tag] = det_pf_cellwise_mu(
            fut_day_temps[tag], ole_mask_idx,
            enhanced_idx_set=ole_enhanced_set,
            mu_enhanced=OLE_MU1, mu_default=OLE_MU0,
            sigma=OLE_SIG
        )

    fut_post_dir = os.path.join(OUT_DIR, "future_post")
    os.makedirs(fut_post_dir, exist_ok=True)
    rail_thr_post_gdf, ole_thr_post_gdf = save_pf_maps_and_thr(
        rail_cells_gdf=rail_cells_gdf,
        ole_cells_gdf=ole_cells_gdf,
        pf_dict_rail=pf_fut_post_rail, pf_dict_ole=pf_fut_post_ole, thr=THR,
        rail_shp=RAIL_SHP, ole_shp=OLE_SHP,
        save_dir=fut_post_dir, save_prefix="future_post"
    )


    rail_len_curr_km  = _length_km_from_thr(curr_thr_rail_gdf, RAIL_SHP)
    rail_len_post_km  = _length_km_from_thr(rail_thr_post_gdf, RAIL_SHP)
    ole_len_curr_km   = _length_km_from_thr(curr_thr_ole_gdf,  OLE_SHP)
    ole_len_post_km   = _length_km_from_thr(ole_thr_post_gdf,  OLE_SHP)

    def _compare(pre_km, post_km):
        if pre_km <= 0:
            return 0.0, 0.0
        delta = pre_km - post_km
        pct = 100.0 * delta / pre_km
        return delta, pct

    rail_delta_km, rail_reduct_pct = _compare(rail_len_curr_km, rail_len_post_km)
    ole_delta_km,  ole_reduct_pct  = _compare(ole_len_curr_km,  ole_len_post_km)

    cmp_df = pd.DataFrame([
        {'asset':'RAIL','baseline_km':rail_len_curr_km,'post_km':rail_len_post_km,'delta_km':rail_delta_km,'reduction_%':rail_reduct_pct},
        {'asset':'OLE', 'baseline_km':ole_len_curr_km, 'post_km':ole_len_post_km, 'delta_km':ole_delta_km, 'reduction_%':ole_reduct_pct}
    ])
    cmp_csv = os.path.join(OUT_DIR, "compare_baseline_vs_future_post_maxday.csv")
    cmp_df.to_csv(cmp_csv, index=False)
    print(f"enhanced compare saved：{cmp_csv}")

if __name__ == "__main__":
    main()

finished time=3600；future time=3600，space=(112,82)
rail grid：1034；ole grid：451


C:\Users\Administrator\AppData\Local\Temp\ipykernel_21804\2551200832.py:300: UserWarning: The GeoDataFrame you are attempting to plot is empty. Nothing has been displayed.
  thr_gdf.plot(ax=ax, color=color, edgecolor='black', linewidth=0.2)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_21804\2551200832.py:300: UserWarning: The GeoDataFrame you are attempting to plot is empty. Nothing has been displayed.
  thr_gdf.plot(ax=ax, color=color, edgecolor='black', linewidth=0.2)


day(summer)：
  current -> current: max=227, med=178, min=128 (day 索引 0-359)
  future -> future: max=239, med=179, min=134 (day 索引 0-359)
material saved：outputs_final_with_shapes5\materials_future_maxday.csv


C:\Users\Administrator\AppData\Local\Temp\ipykernel_21804\2551200832.py:300: UserWarning: The GeoDataFrame you are attempting to plot is empty. Nothing has been displayed.
  thr_gdf.plot(ax=ax, color=color, edgecolor='black', linewidth=0.2)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_21804\2551200832.py:300: UserWarning: The GeoDataFrame you are attempting to plot is empty. Nothing has been displayed.
  thr_gdf.plot(ax=ax, color=color, edgecolor='black', linewidth=0.2)


enhanced compare saved：outputs_final_with_shapes5\compare_baseline_vs_future_post_maxday.csv
